# ML HW5
### 106598018 萬俊瑋

**1. Show the following is true. If 𝐽 = −{𝑑log2 𝑦 + (1 − 𝑑)log2(1 − 𝑦)} and y = 𝑓(𝐱) = 1/(1+exp(−𝒘𝑇𝐱)), then 𝜕𝐽/𝜕𝑤𝑗 = 𝑥𝑗(𝑦 − 𝑑).**

![](https://github.com/wei840222/ML-HW/blob/master/images/HW5Q1.jpg?raw=true)

**2. Compute the complete update (back propagation) equations for all weights (𝑤1~𝑤8) in the following neural networks. This time, the activation function in the hidden layer is ReLU and that in the output layer remains sigmoid. In addition, use softmax as the cost function.**
![](https://github.com/wei840222/ML-HW/blob/master/images/HW4Q4NN.JPG?raw=true)
![](https://github.com/wei840222/ML-HW/blob/master/images/HW5Q2.jpg?raw=true)

**3. Write a program to implement the neural network with your back propagation equations in problem 2. To test your network, train it to distinguish the classes of versicolor and virginica in the Iris dataset using only the third and fourth features (i.e., petal length and petal width) as the inputs. As usual, use 70% of the data for training and the rest for testing. Repeat the experiments 10 times to find the average accuracy. During training, set the desired output as 1.0 for in class data and 0.0 for out of class data. Don’t forget to use random numbers as the initial weights. In addition, monitor the cost (loss) function with respect to the training epochs. Do you observe any difference when compared with the cost function plot using MSE?**

**4. The following shows the LeNet-5 architecture. If we use the weight-sharing approach for the first layer, compute the number of connections and trainable weights from the input layer to the first hidden layer C1. To compute the results, you need the following parameters: kernel size = 5x5, stride = 1, and no zero-padding.**

![](https://github.com/wei840222/ML-HW/blob/master/images/HW5LeNet-5.JPG?raw=true)
![](https://github.com/wei840222/ML-HW/blob/master/images/HW5Q4.jpg?raw=true)

**5. Assuming that the following is a part of convolution neural networks. Compute the resultant values (size of 3x3) if it has two input planes, stride of one, no zero-padding, and using the ReLU activation function.**

![](https://github.com/wei840222/ML-HW/blob/master/images/HW5Q5.jpg?raw=true)